<a href="https://colab.research.google.com/github/mateus-abrantes/MLP-com-Keras/blob/main/MLP_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multilayer Peceptron com Keras aplicado na previsao de chuva na Australia

O objetivo deste exemplo é treinar um modelo supervisionado para prever se ira chover no outro dia em algun lugar da Australia, visto os dados obtidos dos ultimos 10 anos. Sendo que nestes dados apresentam as informações climaticas de quase todos os dias do ano, e a informação se choveu nesse dia, nessa aplicação queremos prever se choveu nesse dia, ira chover no outro dia.

## Referências 
* Kaggle, https://www.kaggle.com/shrutimechlearn/churn-modelling
* Super Data Science, https://www.superdatascience.com/machine-learning/ 


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Carregando os dados

In [107]:
# Importing the dataset
dataset = pd.read_csv('https://raw.githubusercontent.com/mateus-abrantes/MLP-com-Keras/main/weatherAUS.csv')
dataset

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,ENE,13.0,11.0,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,N,13.0,9.0,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,WNW,9.0,9.0,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,N,13.0,7.0,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


In [108]:
#tratando o dataset
#Removendo linhas em que nao existem dados
dataset = dataset.dropna().copy()
#separando o campo data

#Tratamento dos dados do tipo string para numerico
# gender = {'male': 1,'female': 2} 
# dados.sex = [gender[item] for item in dados.sex] 
dataset['Date']= pd.to_datetime(dataset['Date'],format='%Y-%m-%d')
dataset['day'] = dataset['Date'].dt.day
dataset['month'] = dataset['Date'].dt.month
dataset['year'] = dataset['Date'].dt.year
#Removendo campo Date
dataset = dataset.drop("Date", axis=1)
#Organizando colunas
cols = dataset.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
dataset = dataset[cols] 
#Tratamento dos dados do tipo string para numerico
Location = {'Cobar':0, 'CoffsHarbour':1, 'Moree':2, 'NorfolkIsland':3, 'Sydney':4,'SydneyAirport':5, 'WaggaWagga':6, 'Williamtown':7, 'Canberra':8, 'Sale':9,'MelbourneAirport':10, 'Melbourne':11, 'Mildura':12, 'Portland':13, 'Watsonia':14,'Brisbane':15, 'Cairns':16, 'Townsville':17, 'MountGambier':18, 'Nuriootpa':19,'Woomera':20, 'PerthAirport':21, 'Perth':22, 'Hobart':23, 'AliceSprings':24,'Darwin':25}
dataset.Location = [Location[item] for item in dataset.Location] 
WindGustDir = {'SSW':0, 'S':1, 'NNE':2, 'WNW':3, 'N':4, 'SE':5, 'ENE':6, 'NE':7, 'E':8, 'SW':9, 'W':10,'WSW':11, 'NNW':12, 'ESE':13, 'SSE':14, 'NW':15} 
dataset.WindGustDir = [WindGustDir[item] for item in dataset.WindGustDir] 
WindDir9am = {'ENE':0, 'SSE':1, 'NNE':2, 'WNW':3, 'NW':4, 'N':5, 'S':6, 'SE':7, 'NE':8, 'W':9, 'SSW':10,'E':11, 'NNW':12, 'ESE':13, 'WSW':14, 'SW':15} 
dataset.WindDir9am = [WindDir9am[item] for item in dataset.WindDir9am] 
WindDir3pm = {'SW':0, 'SSE':1, 'NNW':2, 'WSW':3, 'WNW':4, 'S':5, 'ENE':6, 'N':7, 'SE':8, 'NNE':9,'NW':10, 'E':11, 'ESE':12, 'NE':13, 'SSW':14, 'W':15} 
dataset.WindDir3pm = [WindDir3pm[item] for item in dataset.WindDir3pm] 
RainToday = {'No':0, 'Yes':1} 
dataset.RainToday = [RainToday[item] for item in dataset.RainToday] 
RainTomorrow = {'No':0, 'Yes':1} 
dataset.RainTomorrow = [RainTomorrow[item] for item in dataset.RainTomorrow] 
dataset

,day,month,year,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
6049,1,1,2009,0,17.9,35.2,0.0,12.0,12.3,0,48.0,0,0,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,0,0
6050,2,1,2009,0,18.4,28.9,0.0,14.8,13.0,1,37.0,1,1,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,0,0
6052,4,1,2009,0,19.4,37.6,0.0,10.8,10.6,2,46.0,2,2,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,0,0
6053,5,1,2009,0,21.9,38.4,0.0,11.4,12.2,3,31.0,3,3,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,0,0
6054,6,1,2009,0,24.2,41.0,0.0,11.2,8.4,3,35.0,4,4,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142298,20,6,2017,25,19.3,33.4,0.0,6.0,11.0,6,35.0,7,13,9.0,20.0,63.0,32.0,1013.9,1010.5,0.0,1.0,24.5,32.3,0,0
142299,21,6,2017,25,21.2,32.6,0.0,7.6,8.6,8,37.0,7,8,13.0,11.0,56.0,28.0,1014.6,1011.2,7.0,0.0,24.8,32.0,0,0
142300,22,6,2017,25,20.7,32.8,0.0,5.6,11.0,8,33.0,11,15,17.0,11.0,46.0,23.0,1015.3,1011.8,0.0,0.0,24.8,32.1,0,0
142301,23,6,2017,25,19.5,31.8,0.0,6.2,10.6,13,26.0,7,2,9.0,17.0,62.0,58.0,1014.9,1010.7,1.0,1.0,24.8,29.2,0,0


In [128]:
X = dataset.iloc[:,0:24].values
y = dataset.iloc[:,24].values

print(X[0:6,:])

[[1.0000e+00 1.0000e+00 2.0090e+03 0.0000e+00 1.7900e+01 3.5200e+01
  0.0000e+00 1.2000e+01 1.2300e+01 0.0000e+00 4.8000e+01 0.0000e+00
  0.0000e+00 6.0000e+00 2.0000e+01 2.0000e+01 1.3000e+01 1.0063e+03
  1.0044e+03 2.0000e+00 5.0000e+00 2.6600e+01 3.3400e+01 0.0000e+00]
 [2.0000e+00 1.0000e+00 2.0090e+03 0.0000e+00 1.8400e+01 2.8900e+01
  0.0000e+00 1.4800e+01 1.3000e+01 1.0000e+00 3.7000e+01 1.0000e+00
  1.0000e+00 1.9000e+01 1.9000e+01 3.0000e+01 8.0000e+00 1.0129e+03
  1.0121e+03 1.0000e+00 1.0000e+00 2.0300e+01 2.7000e+01 0.0000e+00]
 [4.0000e+00 1.0000e+00 2.0090e+03 0.0000e+00 1.9400e+01 3.7600e+01
  0.0000e+00 1.0800e+01 1.0600e+01 2.0000e+00 4.6000e+01 2.0000e+00
  2.0000e+00 3.0000e+01 1.5000e+01 4.2000e+01 2.2000e+01 1.0123e+03
  1.0092e+03 1.0000e+00 6.0000e+00 2.8700e+01 3.4900e+01 0.0000e+00]
 [5.0000e+00 1.0000e+00 2.0090e+03 0.0000e+00 2.1900e+01 3.8400e+01
  0.0000e+00 1.1400e+01 1.2200e+01 3.0000e+00 3.1000e+01 3.0000e+00
  3.0000e+00 6.0000e+00 6.0000e+00 3.7000e+01

## Dividindo para terinamento e teste

In [134]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Tratando os dados para o treinamento

In [135]:
# Normalização 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [136]:
print(X_test[0:6,:])

[[-0.99774357  1.03946794 -0.90767332  0.74871371 -2.00598891 -1.17641005
  -0.30480098  0.08308324  0.60447807  0.83316371 -0.44061749  0.5759566
  -0.96826418  0.39856964  0.02310031 -0.58794917 -0.42723115  1.56997191
   1.68667341 -1.15670615 -0.4984601  -1.3082613  -1.02324863 -0.53112203]
 [ 0.59783005  1.3298255   1.54224404 -1.32686027  0.9732028   0.60103297
  -0.30480098  0.51615759 -0.46042709  0.15510747 -0.44061749  0.79445027
  -0.32258625 -0.5626828  -0.3289104   0.0596692   0.71181199  0.16565441
   0.04220215  0.62933127  0.256675    0.97392023  0.40947993 -0.53112203]
 [-0.99774357 -1.57375003 -0.49935376  1.4405717   1.53472585  2.04878898
   0.38050896  1.32817199  0.60447807 -1.42702377  1.20619954 -0.51651176
   0.10786571  0.99935242 -1.26760565 -1.07366294 -1.12056175 -2.2955206
  -2.57730959  0.27212379  0.63424255  1.78029103  2.22232014  1.88280648]
 [ 0.82576914 -0.41231982  1.9505636   0.05685572 -0.41500695 -0.64604399
  -0.30480098 -0.07931965 -0.77989864

## Treinamento 

In [137]:
import tensorflow as tf


### Construíndo a estrutura da rede

In [152]:
# Inicialização 
classifier = tf.keras.models.Sequential()

# Adicionando a camada de entrada e a primeira camada escondida  
classifier.add( tf.keras.layers.Dense( activation = 'relu', input_dim = 24, units = 26, kernel_initializer = 'uniform'))

# Adicionando a segunda camada escondida 
classifier.add( tf.keras.layers.Dense( activation = 'relu', units = 26, kernel_initializer = 'uniform' ))

# Adicionando a camada de saída 
classifier.add( tf.keras.layers.Dense( activation = 'sigmoid', units = 1, kernel_initializer = 'uniform'))

In [153]:
# Montando a Rede 
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Treinando 

In [154]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 5, epochs = 10)

Epoch 1/10
9028/9028 [==============================] - 13s 1ms/step - loss: 0.3663 - accuracy: 0.8381
Epoch 2/10
9028/9028 [==============================] - 13s 1ms/step - loss: 0.3322 - accuracy: 0.8535
Epoch 3/10
9028/9028 [==============================] - 13s 1ms/step - loss: 0.3244 - accuracy: 0.8588
Epoch 4/10
9028/9028 [==============================] - 12s 1ms/step - loss: 0.3287 - accuracy: 0.8546
Epoch 5/10
9028/9028 [==============================] - 12s 1ms/step - loss: 0.3237 - accuracy: 0.8567
Epoch 6/10
9028/9028 [==============================] - 12s 1ms/step - loss: 0.3242 - accuracy: 0.8580
Epoch 7/10
9028/9028 [==============================] - 12s 1ms/step - loss: 0.3253 - accuracy: 0.8580
Epoch 8/10
9028/9028 [==============================] - 12s 1ms/step - loss: 0.3192 - accuracy: 0.8604
Epoch 9/10
9028/9028 [==============================] - 12s 1ms/step - loss: 0.3202 - accuracy: 0.8619
Epoch 10/10
9028/9028 [==============================] - 12s 1ms/step - l

## Avaliação do treinamento 

In [155]:
# Testando a rede 
# Apresentando o conjunto de testes e usando o método predict 
y_pred = classifier.predict(X_test)
print(y_pred[0:10])

y_pred = (y_pred > 0.5)
print(y_pred[0:10])

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

[[0.002193  ]
 [0.38577113]
 [0.1552599 ]
 [0.32905483]
 [0.02400553]
 [0.00909367]
 [0.34874082]
 [0.00614169]
 [0.00587797]
 [0.1651851 ]]
[[False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]]


In [156]:
print("Matriz de Confusão:")
print(cm)
print("Taxa de acerto:")
print((cm[0,0]+cm[1,1])/len(y_test) )
print(len(y_test))

Matriz de Confusão:
[[8266  588]
 [ 973 1457]]
Taxa de acerto:
0.8616625310173698
11284


## Treinamento com 3 camadas escondidas

In [157]:
import tensorflow as tf


### Construíndo a estrutura da rede

In [158]:
# Inicialização 
classifier = tf.keras.models.Sequential()

# Adicionando a camada de entrada e a primeira camada escondida  
classifier.add( tf.keras.layers.Dense( activation = 'relu', input_dim = 24, units = 26, kernel_initializer = 'uniform'))

# Adicionando a segunda camada escondida 
classifier.add( tf.keras.layers.Dense( activation = 'relu', units = 26, kernel_initializer = 'uniform' ))

# Adicionando a terceira camada escondida 
classifier.add( tf.keras.layers.Dense( activation = 'relu', units = 26, kernel_initializer = 'uniform' ))

# Adicionando a camada de saída 
classifier.add( tf.keras.layers.Dense( activation = 'sigmoid', units = 1, kernel_initializer = 'uniform'))

In [159]:
# Montando a Rede 
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Treinando 

In [160]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 5, epochs = 10)

Epoch 1/10
9028/9028 [==============================] - 13s 1ms/step - loss: 0.3670 - accuracy: 0.8382
Epoch 2/10
9028/9028 [==============================] - 13s 1ms/step - loss: 0.3353 - accuracy: 0.8533
Epoch 3/10
9028/9028 [==============================] - 13s 1ms/step - loss: 0.3335 - accuracy: 0.8542
Epoch 4/10
9028/9028 [==============================] - 13s 1ms/step - loss: 0.3295 - accuracy: 0.8561
Epoch 5/10
9028/9028 [==============================] - 13s 1ms/step - loss: 0.3286 - accuracy: 0.8548
Epoch 6/10
9028/9028 [==============================] - 13s 1ms/step - loss: 0.3291 - accuracy: 0.8566
Epoch 7/10
9028/9028 [==============================] - 13s 1ms/step - loss: 0.3238 - accuracy: 0.8580
Epoch 8/10
9028/9028 [==============================] - 13s 1ms/step - loss: 0.3233 - accuracy: 0.8583
Epoch 9/10
9028/9028 [==============================] - 14s 2ms/step - loss: 0.3173 - accuracy: 0.8622
Epoch 10/10
9028/9028 [==============================] - 13s 1ms/step - l

## Avaliação do treinamento 

In [161]:
# Testando a rede 
# Apresentando o conjunto de testes e usando o método predict 
y_pred = classifier.predict(X_test)
print(y_pred[0:10])

y_pred = (y_pred > 0.5)
print(y_pred[0:10])

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

[[0.0073154 ]
 [0.29296792]
 [0.2417095 ]
 [0.35841274]
 [0.07529622]
 [0.01278964]
 [0.30802655]
 [0.01770312]
 [0.01218089]
 [0.2241509 ]]
[[False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]]


In [162]:
print("Matriz de Confusão:")
print(cm)
print("Taxa de acerto:")
print((cm[0,0]+cm[1,1])/len(y_test) )
print(len(y_test))

Matriz de Confusão:
[[8482  372]
 [1165 1265]]
Taxa de acerto:
0.8637894363700815
11284


# Resultados
Obtivemos uma acurácia bem significativa com uma rede de 2 camadas escondidas obtendo uma taxa de acerto de aproximadamente 86,16%. Já utilizando uma rede com 3 camadas escondidas obtivemos uma taxa de acerto ligeiramente melhor de aproximadamente 86,37%. Com isso obtivemos uma bons resultados, vale salientar que pela grande quantidade de valores nulos, foi retirados várias ocorrências do dataset, o que pode provocar uma má aproximação, já que estamos utilizando datas, poderia ser feito uma analise mais minunciosa tentando interpolar esses valores faltantes, mas que ainda sim poderia causar um certo erro visto que alguns valores seguidos são vazios como os anos iniciais, o que provocaria também uma certa má interpolação.